In [3]:
import requests
import bs4 as bs
import numpy as np
import pandas as pd
import re
from html import unescape


In [2]:
processed_df = pd.read_csv('preprocessed_data.csv') #Final DF to be analyzed (will add new matches to this DF)
fighter_stats = pd.read_csv('data.csv') #compilation of existing fighter stats

In [4]:
#setting up... I will extract these columns from fighter_stats, for the fighters in the new matches...


columns_str = list(processed_df.columns) #all columns in preprocessed data
fighter_str = list(fighter_stats.columns) #all columns in the fighter stats

col_names_R = list(filter(None, [re.match(r'R.*', item) for item in columns_str])) 
col_names_R = [item.group(0) for item in col_names_R]

col_names_B = list(filter(None, [re.match(r'B.*', item) for item in columns_str])) 
col_names_B = [item.group(0) for item in col_names_B]

fighter_R = list(filter(None, [re.match(r'R.*', item) for item in fighter_str])) 
fighter_R = [item.group(0) for item in fighter_R]
fighter_R.remove('Referee') #columns for Fighter_R

fighter_B = list(filter(None, [re.match(r'B.*', item) for item in fighter_str])) 
fighter_B = [item.group(0) for item in fighter_B] #columns for Fighter_B

In [5]:
url = 'http://ufcstats.com/statistics/events/completed'

In [6]:
page = requests.get(url)

In [7]:
soup = bs.BeautifulSoup(page.text, 'html.parser')
soup

<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
    Stats | UFC
  </title>
<meta content="" name="description"/>
<meta content="" name="viewport"/>
<link href="/blocks/main.css?ver=919691" rel="stylesheet"/>
<script src="/js/vendor/modernizr-2.6.2.min.js"></script>
<script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
    })(window,document,'script','//www.google-analytics.com/analytics.js','ga');

    ga('create', 'UA-2855164-1', 'auto');
    ga('send

In [8]:
#List of more urls to scrape:
detail_urls = [] #this grabs urls for each day recorded
for url in soup.find_all(class_='b-link b-link_style_black'):
    detail_urls.append(url['href'])

In [11]:
detail_urls = detail_urls[0:2]
detail_urls

['http://ufcstats.com/event-details/4565d435005319c0',
 'http://ufcstats.com/event-details/b09890ba7ce1d1e2']

In [12]:
def extract_fights_df(url):
    #setup the correct url page
    day = requests.get(url)
    soup = bs.BeautifulSoup(day.text, 'html.parser')
    
    #grab the table using pandas read_html
    a = pd.read_html(str(soup.find(class_='b-fight-details__table b-fight-details__table_style_margin-top b-fight-details__table_type_event-details js-fight-table')))
    df = a[0]
    
    #expand the names... .we'll need to concatenate them manually to match...
    df = df.drop(['Str', 'Td', 'Sub', 'Pass'], axis = 1)
 
    return df

In [13]:
#Let's extract fights from multiple pages:
combined_df = pd.DataFrame()
for url in detail_urls:
    combined_df = pd.concat([combined_df, extract_fights_df(url)], axis = 0)


In [14]:
combined_df = combined_df.reset_index(drop = True)
combined_df

,W/L,Fighter,Weight class,Method,Round,Time
0,win,Kamaru Usman Colby Covington,Welterweight,KO/TKO Punch,5,4:10
1,win,Alexander Volkanovski Max Holloway,Featherweight,U-DEC,5,5:00
2,win,Amanda Nunes Germaine de Randamie,Women's Bantamweight,U-DEC,5,5:00
3,win,Marlon Moraes Jose Aldo,Bantamweight,S-DEC,3,5:00
4,win,Petr Yan Urijah Faber,Bantamweight,KO/TKO Kick,3,0:43
5,win,Geoff Neal Mike Perry,Welterweight,KO/TKO Punches,1,1:30
6,win,Irene Aldana Ketlen Vieira,Women's Bantamweight,KO/TKO Punch,1,4:51
7,win,Omari Akhmedov Ian Heinisch,Middleweight,U-DEC,3,5:00
8,win,Matt Brown Ben Saunders,Welterweight,KO/TKO Punches,2,4:55
9,win,Chase Hooper Daniel Teymur,Featherweight,KO/TKO Elbows,1,4:34


In [15]:
def split_name(name):
    indices = [(m.start(0), m.end(0)) for m in re.finditer(r'\s\s', name)][0]
    name1 = name[:indices[0]]
    name2 = name[indices[1]:]
    return [name1, name2]

In [16]:
#splits the names in the Fighter column

winner = []
loser = []

for item in combined_df['Fighter']:
    names = split_name(item)
    winner.append(names[0])
    loser.append(names[1])
    

In [17]:
combined_df['R_fighter'] = winner
combined_df['B_fighter'] = loser

In [18]:
combined_df


,W/L,Fighter,Weight class,Method,Round,Time,R_fighter,B_fighter
0,win,Kamaru Usman Colby Covington,Welterweight,KO/TKO Punch,5,4:10,Kamaru Usman,Colby Covington
1,win,Alexander Volkanovski Max Holloway,Featherweight,U-DEC,5,5:00,Alexander Volkanovski,Max Holloway
2,win,Amanda Nunes Germaine de Randamie,Women's Bantamweight,U-DEC,5,5:00,Amanda Nunes,Germaine de Randamie
3,win,Marlon Moraes Jose Aldo,Bantamweight,S-DEC,3,5:00,Marlon Moraes,Jose Aldo
4,win,Petr Yan Urijah Faber,Bantamweight,KO/TKO Kick,3,0:43,Petr Yan,Urijah Faber
5,win,Geoff Neal Mike Perry,Welterweight,KO/TKO Punches,1,1:30,Geoff Neal,Mike Perry
6,win,Irene Aldana Ketlen Vieira,Women's Bantamweight,KO/TKO Punch,1,4:51,Irene Aldana,Ketlen Vieira
7,win,Omari Akhmedov Ian Heinisch,Middleweight,U-DEC,3,5:00,Omari Akhmedov,Ian Heinisch
8,win,Matt Brown Ben Saunders,Welterweight,KO/TKO Punches,2,4:55,Matt Brown,Ben Saunders
9,win,Chase Hooper Daniel Teymur,Featherweight,KO/TKO Elbows,1,4:34,Chase Hooper,Daniel Teymur


In [19]:
def extract_recent_fight(fighter): #This function returns all the stats from fighter's most recent fight

    try:
        R_df = pd.DataFrame(fighter_stats[fighter_stats['R_fighter'] == fighter])
    except:
        print('Fighter not in list R')
    
    try: 
        B_df = pd.DataFrame(fighter_stats[fighter_stats['B_fighter'] == fighter])
    except:
        print('Fighter not in list B')
        
    combined_df = pd.concat([R_df, B_df], axis = 0)
    combined_df = combined_df.sort_values('date', ascending = False)
        
    return combined_df.iloc[0, ].copy()
    

In [20]:
def recent_fight_stats(fighter): #This function returns the stats for only the fighter's most recent stats:
    fight = extract_recent_fight(fighter)
    if fight['R_fighter'] == fighter:
        return fight[fighter_R]
        
    if fight['B_fighter'] == fighter:    
        return fight[fighter_B]

In [22]:
#These functions take the fighter's most recent stats and changes the colnames to match the final template:

def name_columns_R(fighter):
    data = pd.DataFrame(recent_fight_stats(fighter)).transpose()
    data.columns = fighter_R
    
    return data
    
def name_columns_B(fighter):
    data = pd.DataFrame(recent_fight_stats(fighter)).transpose()
    data.columns = fighter_B
    
    return data
    
def remove_space(text):
    new_text = unescape(text)
    return re.sub(r'\s\s+'," ", new_text)

In [23]:
fighter_R_stats = pd.DataFrame()
fighter_B_stats = pd.DataFrame()

for (r, b) in zip(combined_df['R_fighter'], combined_df['B_fighter']):
    try:
        fighter_R_stats = pd.concat([fighter_R_stats, pd.DataFrame(name_columns_R(r))])
    except:
        fighter_R_stats = fighter_R_stats.append(pd.Series(), ignore_index = True)
        
    try:
        fighter_B_stats = pd.concat([fighter_B_stats, pd.DataFrame(name_columns_B(b))])
    except:
        fighter_B_stats = fighter_B_stats.append(pd.Series(), ignore_index = True)

In [24]:
fighter_R_stats = fighter_R_stats.reset_index(drop = True)
fighter_B_stats = fighter_B_stats.reset_index(drop = True)

In [25]:
combined_template = pd.concat([fighter_R_stats, fighter_B_stats], axis = 1)

In [26]:
combined_template.shape

(25, 138)

In [27]:
combined_template = pd.concat([combined_df, combined_template], axis = 1)
combined_template

,W/L,Fighter,Weight class,Method,Round,Time,R_fighter,B_fighter,R_fighter,R_current_lose_streak,...,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,B_age
0,win,Kamaru Usman Colby Covington,Welterweight,KO/TKO Punch,5,4:10,Kamaru Usman,Colby Covington,Kamaru Usman,0,...,4,2,2,0,8,Orthodox,180.34,182.88,170,30
1,win,Alexander Volkanovski Max Holloway,Featherweight,U-DEC,5,5:00,Alexander Volkanovski,Max Holloway,Alexander Volkanovski,0,...,4,8,2,1,16,Orthodox,180.34,175.26,155,27
2,win,Amanda Nunes Germaine de Randamie,Women's Bantamweight,U-DEC,5,5:00,Amanda Nunes,Germaine de Randamie,Amanda Nunes,0,...,1,2,0,0,4,Orthodox,175.26,180.34,135,34
3,win,Marlon Moraes Jose Aldo,Bantamweight,S-DEC,3,5:00,Marlon Moraes,Jose Aldo,Marlon Moraes,0,...,6,4,0,0,10,Orthodox,170.18,177.8,145,32
4,win,Petr Yan Urijah Faber,Bantamweight,KO/TKO Kick,3,0:43,Petr Yan,Urijah Faber,Petr Yan,0,...,3,0,6,0,9,Orthodox,167.64,170.18,135,37
5,win,Geoff Neal Mike Perry,Welterweight,KO/TKO Punches,1,1:30,Geoff Neal,Mike Perry,Geoff Neal,0,...,0,4,0,0,5,Orthodox,177.8,180.34,170,27
6,win,Irene Aldana Ketlen Vieira,Women's Bantamweight,KO/TKO Punch,1,4:51,Irene Aldana,Ketlen Vieira,Irene Aldana,0,...,1,0,1,0,3,Orthodox,172.72,172.72,135,26
7,win,Omari Akhmedov Ian Heinisch,Middleweight,U-DEC,3,5:00,Omari Akhmedov,Ian Heinisch,Omari Akhmedov,1,...,1,0,0,0,1,Orthodox,180.34,182.88,185,30
8,win,Matt Brown Ben Saunders,Welterweight,KO/TKO Punches,2,4:55,Matt Brown,Ben Saunders,Matt Brown,3,...,2,3,3,0,9,Orthodox,187.96,195.58,170,36
9,win,Chase Hooper Daniel Teymur,Featherweight,KO/TKO Elbows,1,4:34,Chase Hooper,Daniel Teymur,NaN,NaN,...,0,0,0,0,0,Orthodox,165.1,175.26,145,31


In [28]:
combined_template.dropna(how = 'all', inplace = True)
combined_template.drop(combined_df.columns[0], axis=1, inplace = True)
combined_template

,Fighter,Weight class,Method,Round,Time,R_fighter,B_fighter,R_fighter,R_current_lose_streak,R_current_win_streak,...,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,B_age
0,Kamaru Usman Colby Covington,Welterweight,KO/TKO Punch,5,4:10,Kamaru Usman,Colby Covington,Kamaru Usman,0,9,...,4,2,2,0,8,Orthodox,180.34,182.88,170,30
1,Alexander Volkanovski Max Holloway,Featherweight,U-DEC,5,5:00,Alexander Volkanovski,Max Holloway,Alexander Volkanovski,0,6,...,4,8,2,1,16,Orthodox,180.34,175.26,155,27
2,Amanda Nunes Germaine de Randamie,Women's Bantamweight,U-DEC,5,5:00,Amanda Nunes,Germaine de Randamie,Amanda Nunes,0,7,...,1,2,0,0,4,Orthodox,175.26,180.34,135,34
3,Marlon Moraes Jose Aldo,Bantamweight,S-DEC,3,5:00,Marlon Moraes,Jose Aldo,Marlon Moraes,0,4,...,6,4,0,0,10,Orthodox,170.18,177.8,145,32
4,Petr Yan Urijah Faber,Bantamweight,KO/TKO Kick,3,0:43,Petr Yan,Urijah Faber,Petr Yan,0,4,...,3,0,6,0,9,Orthodox,167.64,170.18,135,37
5,Geoff Neal Mike Perry,Welterweight,KO/TKO Punches,1,1:30,Geoff Neal,Mike Perry,Geoff Neal,0,2,...,0,4,0,0,5,Orthodox,177.8,180.34,170,27
6,Irene Aldana Ketlen Vieira,Women's Bantamweight,KO/TKO Punch,1,4:51,Irene Aldana,Ketlen Vieira,Irene Aldana,0,2,...,1,0,1,0,3,Orthodox,172.72,172.72,135,26
7,Omari Akhmedov Ian Heinisch,Middleweight,U-DEC,3,5:00,Omari Akhmedov,Ian Heinisch,Omari Akhmedov,1,0,...,1,0,0,0,1,Orthodox,180.34,182.88,185,30
8,Matt Brown Ben Saunders,Welterweight,KO/TKO Punches,2,4:55,Matt Brown,Ben Saunders,Matt Brown,3,0,...,2,3,3,0,9,Orthodox,187.96,195.58,170,36
9,Chase Hooper Daniel Teymur,Featherweight,KO/TKO Elbows,1,4:34,Chase Hooper,Daniel Teymur,NaN,NaN,NaN,...,0,0,0,0,0,Orthodox,165.1,175.26,145,31


In [29]:
combined_template.to_csv('new_fights.csv')

In [93]:
combined_template = pd.read_csv('new_fights.csv')

In [94]:
combined_template.fillna(0, inplace=True)

In [95]:
method_dict = {
    'CNC': 'R_draw',
    'KO/TKO': 'R_win_by_KO/TKO',
    'KO/TKO  Elbow': 'R_win_by_KO/TKO',
    'KO/TKO  Elbows': 'R_win_by_KO/TKO',
    'KO/TKO  Kick': 'R_win_by_KO/TKO',
    'KO/TKO  Knee': 'R_win_by_KO/TKO',
    'KO/TKO  Punch': 'R_win_by_KO/TKO',
    'KO/TKO  Punches': 'R_win_by_KO/TKO',
    'KO/TKO  Spinning Back Fist': 'R_win_by_KO/TKO',
    'M-DEC': 'R_win_by_Decision_Majority',
    'S-DEC': 'R_win_by_Decision_Split',
    'SUB  Anaconda Choke': 'R_win_by_Submission',
    'SUB  Arm Triangle': 'R_win_by_Submission',
    'SUB  Armbar': 'R_win_by_Submission',
    'SUB  Guillotine Choke': 'R_win_by_Submission',
    'SUB  Other - Choke': 'R_win_by_Submission',
    'SUB  Other': 'R_win_by_Submission',
    'SUB  Peruvian Necktie': 'R_win_by_Submission',
    'SUB  Rear Naked Choke': 'R_win_by_Submission',
    'SUB  Triangle Choke': 'R_win_by_Submission',
    'U-DEC': 'R_win_by_Decision_Unanimous'
}

In [96]:
def calc_seconds(time):
    split_time = time.split(":", )
    return int(split_time[0])*60 + int(split_time[1])

In [97]:
#update relevant values for matches:
for row in np.arange(0,combined_template.shape[0])[::-1]:
    combined_template.R_current_win_streak[row] += 1
    combined_template.R_current_lose_streak[row] = 0
    combined_template.B_current_lose_streak[row] += 1
    combined_template.B_current_win_streak[row] = 0   
    combined_template.R_wins[row] += 1
    combined_template.R_total_rounds_fought[row] += combined_template.Round[row]
    combined_template.B_total_rounds_fought[row] += combined_template.Round[row]
    combined_template[method_dict[combined_template.Method[row]]][row] += 1 #additional win under the correct format
    combined_template['R_total_time_fought(seconds)'][row] += calc_seconds(combined_template.Time[row])    
    combined_template['B_total_time_fought(seconds)'][row] += calc_seconds(combined_template.Time[row])    


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda\lib\s

AttributeError: 'DataFrame' object has no attribute 'Round'

In [98]:
weight_class_dict = {
    'Bantamweight': "weight_class_Bantamweight",
    'Catch Weight': "weight_class_Catch Weight",
    'Featherweight': "weight_class_Featherweight",
    'Flyweight': "weight_class_Flyweight",
    'Heavyweight': "weight_class_Heavyweight",
    'Light Heavyweight': "weight_class_Light Heavyweight",
    'Lightweight': "weight_class_Lightweight",
    'Middleweight': "weight_class_Middleweight",
    'Open Weight': "weight_class_Open Weight",
    'Welterweight': "weight_class_Welterweight",
    "Women's Bantamweight": "weight_class_Women's Bantamweight",
    "Women's Featherweight": "weight_class_Women's Featherweight",
    "Women's Flyweight": "weight_class_Women's Flyweight",
    "Women's Strawweight": "weight_class_Women's Strawweight"   
}

weightcols = list(['Bantamweight',
 'Catch Weight',
 'Featherweight',
 'Flyweight',
 'Heavyweight',
 'Light Heavyweight',
 'Lightweight',
 'Middleweight',
 'Welterweight',
 "Women's Bantamweight",
 "Women's Featherweight",
 "Women's Flyweight",
 "Women's Strawweight"])

In [99]:
from sklearn.preprocessing import OneHotEncoder
weight_enc = OneHotEncoder()
weight_enc.fit(np.array(combined_template['Weight class']).reshape(-1,1))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [100]:
a = weight_enc.transform(np.array(combined_template['Weight class']).reshape(-1,1))
weightdf = pd.DataFrame(a.toarray())
weight_cols = [weight_class_dict[weight] for weight in list(weight_enc.categories_[0])] #match columns with existing fight columns
weightdf.columns = weight_cols

In [103]:
combined_template['R_Stance'] = combined_template['R_Stance'].fillna("")
combined_template['B_Stance'] = combined_template['B_Stance'].fillna("")

stance_enc = OneHotEncoder()
stance_r = stance_enc.fit_transform(np.array(combined_template['R_Stance']).reshape(-1,1))
stance_b = stance_enc.fit_transform(np.array(combined_template['B_Stance']).reshape(-1,1))

In [109]:
stancedf_R = pd.DataFrame(stance_r.toarray())
stancedf_R.columns = list(['R_Stance_Orthodox', 'R_Stance_Southpaw', 'R_Stance_Switch'])
stancedf_R

stancedf_B = pd.DataFrame(stance_b.toarray())
stancedf_B.columns = list(['B_Stance_Orthodox'])
stancedf_B

,B_Stance_Orthodox
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0


In [110]:
encoded_template = pd.concat([combined_template, weightdf], axis = 1)
encoded_template = pd.concat([encoded_template, stancedf_R, stancedf_B], axis = 1)

In [111]:
encoded_template.to_csv('new_encoded_template.csv')

In [120]:
encoded_template = pd.read_csv('new_encoded_template.csv')

In [121]:
test = pd.concat([encoded_template, processed_df], axis = 0, sort = False)

In [122]:
rearrange_cols = processed_df.columns #match columns in processed_df

In [123]:
test = pd.concat([encoded_template[rearrange_cols], processed_df], axis = 0, sort = True)

In [124]:
test = test[rearrange_cols]

In [125]:
test.to_csv('new_fights_final.csv')